<a href="https://colab.research.google.com/github/Tam1979/TATA-ML/blob/master/w2_2a_dec_tree_prosper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Tree Learning: Prosper Loan Dataset

A decision tree a learned set of rules that allows us to make decisions on data.

We are going to look at the prosper loan dataset.  This dataset shows a history of loans made by Prosper.

In [0]:
%matplotlib inline
import pandas as pd
import time

## Step 1: Load the Data

In [0]:
## small file, start with this
datafile = "https://s3.amazonaws.com/elephantscale-public/data/prosper-loan/prosper-loan-data-sample.csv"
## this is a large file
#datafile = "https://s3.amazonaws.com/elephantscale-public/data/prosper-loan/prosper-loan-data.csv.gz"

t1 = time.perf_counter()
data = pd.read_csv(datafile)
t2 = time.perf_counter()

print("read {:,} records in {:,.2f} ms".format(len(data), (t2-t1)*1000))
# schema
data.dtypes

read 200 records in 401.79 ms


Term                                     int64
LoanStatus                               int64
BorrowerRate                           float64
ProsperRating (numeric)                float64
ProsperScore                           float64
ListingCategory                         object
BorrowerState                           object
EmploymentStatus                        object
EmploymentStatusDuration               float64
IsBorrowerHomeowner                       bool
CreditScore                            float64
CurrentCreditLines                     float64
OpenCreditLines                        float64
TotalCreditLinespast7years             float64
OpenRevolvingAccounts                    int64
OpenRevolvingMonthlyPayment            float64
InquiriesLast6Months                   float64
TotalInquiries                         float64
CurrentDelinquencies                   float64
AmountDelinquent                       float64
DelinquenciesLast7Years                float64
PublicRecords

In [0]:
data

,Term,LoanStatus,BorrowerRate,ProsperRating (numeric),ProsperScore,ListingCategory,BorrowerState,EmploymentStatus,EmploymentStatusDuration,IsBorrowerHomeowner,...,ProsperPaymentsOneMonthPlusLate,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,LoanOriginalAmount,MonthlyLoanPayment,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors,YearsWithCredit
0,36,1,0.0789,7.0,9.0,Debt,VA,Employed,9.0,True,...,0.0,0.0,0.00,17300,541.24,0,0,0.0,169,20
1,36,0,0.1450,4.0,6.0,Unknown,OR,Part-time,15.0,False,...,0.0,0.0,0.00,9500,327.00,0,0,0.0,171,23
2,36,1,0.1707,4.0,6.0,Debt,UT,Full-time,22.0,True,...,0.0,0.0,0.00,7200,256.95,0,0,0.0,216,16
3,36,1,0.0830,7.0,9.0,Other,VA,Full-time,175.0,True,...,0.0,0.0,0.00,4500,141.64,0,0,0.0,151,18
4,36,1,0.1550,4.0,6.0,Debt,IL,Full-time,18.0,False,...,0.0,0.0,0.00,8000,277.14,0,0,0.0,140,14
5,36,1,0.2099,4.0,6.0,Other,IN,Employed,77.0,True,...,0.0,0.0,0.00,4000,150.68,0,0,0.0,74,22
6,36,1,0.0950,7.0,9.0,Auto,MI,Full-time,43.0,True,...,0.0,10000.0,5584.95,6000,192.20,0,0,0.0,216,21
7,36,1,0.0670,4.0,6.0,Business,PA,Full-time,40.0,False,...,0.0,0.0,0.00,1250,38.43,0,0,0.0,70,14
8,36,1,0.1330,4.0,6.0,Unknown,MD,Self-employed,5.0,True,...,0.0,0.0,0.00,8000,270.71,0,0,0.0,242,24
9,36,1,0.1000,4.0,6.0,Personal,NY,Full-time,42.0,True,...,0.0,0.0,0.00,25000,797.25,0,0,0.0,730,30


In [0]:
## TODO : select a few columns 
## start with: 'LoanStatus',  'EmploymentStatus', 'CreditScore', 'StatedMonthlyIncome', 'ListingCategory'
## we can add more later

select_columns = ['LoanStatus', 'EmploymentStatus', 'CreditScore', 'RevolvingCreditBalance', 'Recommendations']

## Note : vector columns can only have Numbers, don't include Categorical columns here
## And definitely not 'LoanStatus'  (if you are curiuos include and see what happens!)
vector_columns = [ 'EmpIndex', 'CreditScore', 'StatedMonthlyIncome', 'CategoryIndex']



In [0]:
## TODO : Extract only the columns we are interested in
## Hint : 'select_columns'
prosper = ['LoanStatus', 'EmploymentStatus', 'CreditScore', 'RevolvingCreditBalance', 'Recommendations']
prosper

['LoanStatus',
 'EmploymentStatus',
 'CreditScore',
 'RevolvingCreditBalance',
 'Recommendations']

## Step 2 : Clean Data

In [0]:
## TODO :  Drop any NA, null values.  
## Hint : Using `.na.drop('LoanStatus', 'EmploymentStatus', 'CreditScore', 'RevolvingCreditBalance', 'Recommendations')`
prosper_clean = prosper.dropna('CreditScore')

print("Original record count {:,}, cleaned records count {:,},  dropped {:,}"\
      .format(len(prosper), len(prosper_clean), 
              (len(prosper) - len(prosper_clean))))
prosper_clean

AttributeError: ignored

## Look at some summary data

**=> Q : What does that say about the cardinality of these categorical columns?**


In [0]:
## TODO : Look at some summaries
## We are going to group counts by 'LoanStatus',  'EmploymentStatus' ,   'ListingCategory'

print(prosper_clean.groupby('LoanStatus').size())
print(prosper_clean.groupby('EmploymentStatus').size())
print(prosper_clean.groupby('CreditScore').size())

## Question : What does that say about the cardinality of these categorical columns? *

NameError: ignored

## Step 3: Converting Categorical columns 

Convert categorical columns to numeric.   
Here let's convert **EmploymentStatus** column

In [0]:
# use pd.factorize on EmploymentStatus

data['EmploymentStatusFactor'] = pd.factorize(data['EmploymentStatus'])[0]


In [0]:
## use pd.factorize on ListingCategory

data['ListingCategoryFactor'] = pd.factorize(data['ListingCategory'])[0]


## Step 4: Build feature vectors 

TODO: Build feature Vectors out of the numeric columns.

In [0]:
features = 1
label = 2

## Step 5: Split Data into training and test.

We will split our the data up into training and test.  (You know the drill by now).

**=> TODO: Split dataset into 70% training, 30% validation**


In [0]:
## TODO :  Split the data into 70% training and 30% test sets 
## Hint : 0.7   , 0.3
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(features, label, test_size = 3)
print("training set x = " , len(train_x))
print("training set y = " , len(train_y))
print("test set x = " , len(test_x))
print("testing set y = " , (len(test_y))

SyntaxError: ignored

## Step 6: Decision Tree
** Q : How many nodes does the tree have? **

In [0]:
from sklearn.tree import DecisionTreeClassifier
##TODO: Create a DecisionTree model with 5000 Maxbins
dt = DecisionTreeClassifier()

t1 = time.perf_counter()
## TODO : train with training data set
## Hint : training
tree_model = dt.fit(train_x, train_y)
t2 = time.perf_counter()

## TODO : Notice the time took to train
print("traind on {:,} records using {:,} features in {:,.2f} ms".\
      format(len(train_x), len(train_y), (t2-t1)*1000))


NameError: ignored

In [0]:
## TODO : create predictions using test dataset
## Hint : test
predictions = tree_model.predict(???)


SyntaxError: ignored

## Step 7: Print out the Tree

In [0]:
from sklearn import tree
from sklearn.externals.six import StringIO  
import pydot 
dot_data = StringIO() 
tree.export_graphviz(tree_model, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph[0].write_png('tree.png')

NameError: ignored

In [0]:
# View the Image
from IPython.display import Image
Image("tree.png")

## Step 7: Evaluate the model.

Let us check to see how the model did, using accuracy as a measure.

In [0]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)


NameError: ignored

## Step 8: Improve Accuracy

### Add more data
In Step-1 change the 'datafile' to the full dataset.  
And see how the accuracy above changes

### Add more features
Look at the schema of the full dataset.  Are there any columns you want to add